
# HIT140 — Assignment 2 Pipeline (Investigation A)

This notebook performs data cleaning **exactly per D1** and continues with the analyses required for **Assignment 2**, using both `dataset1.csv` and `dataset2.csv`.

**Sections**
1. Setup & imports  
2. Load data  
3. D1 Cleaning — datetime strategy, `habit` recode, derived features, outliers (no dropping), season labels, chronology checks  
4. Align with `dataset2` (30‑min windows) and enrich `dataset1`  
5. Descriptive analyses & figures  
6. Inferential analysis (GLM Logit) for **Investigation A**  
7. Save cleaned/enriched outputs  


In [ ]:

# 1) Setup & imports
import pandas as pd
import numpy as np
import re
from pathlib import Path

# Stats/ML
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Viz
import matplotlib.pyplot as plt

OUT_DIR = Path("./assign2_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

pd.options.display.max_columns = 200
pd.options.display.width = 160


In [ ]:

# 2) Load data (ensure the CSVs are in the same folder as this notebook)
d1 = pd.read_csv("dataset1.csv")
d2 = pd.read_csv("dataset2.csv")

print("dataset1 shape:", d1.shape)
print("dataset2 shape:", d2.shape)
display(d1.head())
display(d2.head())


## 3. D1 Cleaning — Datetime strategy & derived fields

In [ ]:

# Parse as day-first and keep full datetimes
def parse_dayfirst(s):
    return pd.to_datetime(s, dayfirst=True, errors="coerce")

for col in ["start_time", "rat_period_start", "rat_period_end", "sunset_time"]:
    d1[col] = parse_dayfirst(d1[col])

d2["time"] = parse_dayfirst(d2["time"])

# Derived fields instead of discarding dates
d1["date"] = d1["start_time"].dt.date
d1["minutes_after_sunset"] = (d1["start_time"] - d1["sunset_time"]).dt.total_seconds() / 60.0
d1["minutes_after_rat_arrival"] = d1["seconds_after_rat_arrival"] / 60.0

# Verify chronology per row: rat_period_start ≤ start_time ≤ rat_period_end
d1["chronology_ok"] = (
    (d1["rat_period_start"] <= d1["start_time"]) &
    (d1["start_time"] <= d1["rat_period_end"])
)
violations = (~d1["chronology_ok"]).sum()
print("Chronology violations:", violations)
d1.loc[~d1["chronology_ok"]].head()

### D1 Cleaning — `habit` recode (Unknowns, synonyms, order‑invariant composites, rare→Other)

In [ ]:

def normalize_token(t):
    t = t.strip().lower()
    t = re.sub(r"[\s\-/]+", "_", t)           # spaces/dashes/slashes -> underscore
    t = re.sub(r"[^a-z0-9_]", "", t)            # drop other symbols
    return t

def is_coord_like(s):
    return bool(re.search(r"\d+(\.\d+)?\s*,\s*\d+(\.\d+)?", s))

def clean_habit_value(x):
    if pd.isna(x):
        return "Unknown"
    s = str(x).strip()
    if s == "" or s.isnumeric() or is_coord_like(s):
        return "Unknown"
    s = normalize_token(s)
    # Canonicalize synonyms / spelling
    s = s.replace("bats", "bat")
    s = s.replace("pick_and_bat", "pick_bat")
    s = s.replace("pickandbat", "pick_bat")
    s = s.replace("batfight", "bat_fight")
    s = s.replace("ratpick", "rat_pick")
    s = s.replace("pickbat", "pick_bat")
    s = s.replace("batrat", "bat_rat")
    s = s.replace("ratbat", "rat_bat")
    # Make composite labels order-invariant
    s = s.replace("_and_", "_")
    parts = [p for p in s.split("_") if p != ""]
    if len(parts) > 1:
        s = "_".join(sorted(parts))
    return s or "Unknown"

d1["habit_clean_raw"] = d1["habit"].apply(clean_habit_value)

# Consolidate rare categories (frequency < ~10) into "Other"
freq = d1["habit_clean_raw"].value_counts(dropna=False)
rare_threshold = 10
rare_labels = set(freq[freq < rare_threshold].index.tolist())

keep_labels = {"fast", "pick", "rat", "bat", "bat_fight", "rat_pick", "Unknown", "other"}

def map_to_final_habit(lbl):
    if lbl in keep_labels:
        return lbl
    if lbl in rare_labels:
        return "Other"
    return lbl

d1["habit_clean"] = d1["habit_clean_raw"].apply(map_to_final_habit)

# Show mapping preview
display(pd.DataFrame({
    "original": d1["habit"].head(20),
    "clean_raw": d1["habit_clean_raw"].head(20),
    "clean_final": d1["habit_clean"].head(20)
}))


### D1 Cleaning — Outliers (keep rows; add transforms)

In [ ]:

# Keep original; add transforms for skew reduction
d1["bat_landing_to_food_log1p"] = np.log1p(d1["bat_landing_to_food"].clip(lower=0))
d1["bat_landing_to_food_sqrt"] = np.sqrt(d1["bat_landing_to_food"].clip(lower=0))

d1[["bat_landing_to_food","bat_landing_to_food_log1p","bat_landing_to_food_sqrt"]].describe()


### D1 Cleaning — Season labels

In [ ]:

season_map = {0: "Winter", 1: "Spring"}
if "season" in d1.columns:
    if np.issubdtype(d1["season"].dtype, np.number):
        d1["season_label"] = d1["season"].map(season_map).fillna(d1["season"].astype(str))
    else:
        d1["season_label"] = d1["season"].astype(str)
else:
    d1["season_label"] = d1["month"].map(season_map).fillna(d1["month"].astype(str))

d1["season_label"].value_counts(dropna=False).to_frame("count")


## 4. Enrich `dataset1` with 30‑min context from `dataset2`

In [ ]:

# Build 30-min windows for dataset2
d2 = d2.sort_values("time").copy()
d2["window_start"] = d2["time"]
d2["window_end"] = d2["time"] + pd.Timedelta(minutes=30)

# Merge_asof to align each bat landing time to its window_start (backward), then filter within window
d1 = d1.sort_values("start_time").copy()
enriched = pd.merge_asof(
    d1,
    d2[["window_start","window_end","bat_landing_number","food_availability","rat_minutes","rat_arrival_number"]],
    left_on="start_time",
    right_on="window_start",
    direction="backward"
)
enriched = enriched[(enriched["start_time"] >= enriched["window_start"]) & (enriched["start_time"] < enriched["window_end"])]
print("enriched shape:", enriched.shape)
display(enriched.head())


## 5. Descriptive analyses & figures

In [ ]:

# Numeric summary
desc_cols = [
    "bat_landing_to_food", "bat_landing_to_food_log1p", "bat_landing_to_food_sqrt",
    "seconds_after_rat_arrival", "minutes_after_rat_arrival",
    "hours_after_sunset", "minutes_after_sunset",
    "rat_arrival_number", "bat_landing_number", "food_availability",
    "risk", "reward"
]
summary = enriched[desc_cols].describe().T
display(summary)
summary.to_csv(OUT_DIR / "summary_numeric.csv")

# Habit frequencies
habit_counts = enriched["habit_clean"].value_counts(dropna=False).rename_axis("habit_clean").reset_index(name="n")
display(habit_counts.head(10))
habit_counts.to_csv(OUT_DIR / "habit_counts.csv", index=False)

# Season counts
season_counts = enriched["season_label"].value_counts(dropna=False).rename_axis("season_label").reset_index(name="n")
display(season_counts)
season_counts.to_csv(OUT_DIR / "season_counts.csv", index=False)


In [ ]:

# Figures (matplotlib only — one per figure — no explicit colors)

# Latency distributions
plt.figure()
plt.hist(enriched["bat_landing_to_food"].dropna(), bins=40)
plt.title("Distribution: bat_landing_to_food (seconds)")
plt.xlabel("Seconds from landing to food")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(OUT_DIR / "hist_bat_landing_to_food.png", dpi=180)
plt.show()

plt.figure()
plt.hist(enriched["bat_landing_to_food_log1p"].dropna(), bins=40)
plt.title("Distribution: log1p(bat_landing_to_food)")
plt.xlabel("log1p(seconds)")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(OUT_DIR / "hist_bat_landing_to_food_log1p.png", dpi=180)
plt.show()

# Risk-taking by season
risk_by_season = enriched.groupby("season_label")["risk"].mean().reset_index()
risk_by_season.to_csv(OUT_DIR / "risk_by_season.csv", index=False)

plt.figure()
plt.bar(risk_by_season["season_label"], risk_by_season["risk"])
plt.title("Risk-taking rate by season")
plt.xlabel("Season")
plt.ylabel("Mean risk (1=risk-taking)")
plt.tight_layout()
plt.savefig(OUT_DIR / "bar_risk_by_season.png", dpi=180)
plt.show()

# Risk vs rat pressure (quartiles of rat_arrival_number)
enriched["rat_arrival_bin"] = pd.qcut(enriched["rat_arrival_number"].fillna(0), q=4, duplicates="drop")
risk_by_ratbin = enriched.groupby("rat_arrival_bin")["risk"].mean().reset_index()
risk_by_ratbin.to_csv(OUT_DIR / "risk_by_ratarrival_bin.csv", index=False)

plt.figure()
x = risk_by_ratbin["rat_arrival_bin"].astype(str)
y = risk_by_ratbin["risk"]
plt.bar(x, y)
plt.title("Risk-taking vs. rat_arrival_number (quartiles)")
plt.xlabel("rat_arrival_number quartiles")
plt.ylabel("Mean risk")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig(OUT_DIR / "bar_risk_vs_ratarrival_bins.png", dpi=180)
plt.show()

# Risk vs minutes_after_rat_arrival (quartiles)
enriched["mar_bin"] = pd.qcut(enriched["minutes_after_rat_arrival"], q=4, duplicates="drop")
risk_by_marbin = enriched.groupby("mar_bin")["risk"].mean().reset_index()
risk_by_marbin.to_csv(OUT_DIR / "risk_by_minutes_after_rat_arrival_bin.csv", index=False)

plt.figure()
x = risk_by_marbin["mar_bin"].astype(str)
y = risk_by_marbin["risk"]
plt.bar(x, y)
plt.title("Risk-taking vs. minutes_after_rat_arrival (quartiles)")
plt.xlabel("minutes_after_rat_arrival quartiles")
plt.ylabel("Mean risk")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig(OUT_DIR / "bar_risk_vs_minutes_after_rat_arrival_bins.png", dpi=180)
plt.show()


## 6. Inferential analysis — GLM Logit for Investigation A

In [ ]:

# Drop rows with missing predictors
model_df = enriched.dropna(subset=[
    "risk", "rat_arrival_number", "minutes_after_rat_arrival",
    "minutes_after_sunset", "bat_landing_to_food_log1p", "season_label", "habit_clean"
]).copy()

# Categorical types
model_df["season_label"] = model_df["season_label"].astype("category")
model_df["habit_clean"] = model_df["habit_clean"].astype("category")

# Model: does rat pressure/time since rat arrival predict risk-taking?
formula = (
    "risk ~ rat_arrival_number + minutes_after_rat_arrival + minutes_after_sunset "
    "+ bat_landing_to_food_log1p + C(season_label) + C(habit_clean)"
)

try:
    logit_model = smf.glm(formula=formula, data=model_df, family=sm.families.Binomial()).fit()
    print(logit_model.summary())
    # Save text and odds ratios
    with open(OUT_DIR / "logistic_results.txt", "w") as f:
        f.write(logit_model.summary().as_text())
    params = logit_model.params
    conf = logit_model.conf_int()
    or_table = pd.DataFrame({
        "OR": np.exp(params),
        "CI_low": np.exp(conf[0]),
        "CI_high": np.exp(conf[1]),
        "pvalue": logit_model.pvalues
    })
    display(or_table)
    or_table.to_csv(OUT_DIR / "logistic_odds_ratios.csv")
except Exception as e:
    print("Model fitting failed:", e)


## 7. Save cleaned/enriched datasets and a compact data dictionary

In [ ]:

# Save cleaned/intermediate/final tables
d1.to_csv(OUT_DIR / "dataset1_cleaned.csv", index=False)
d2.to_csv(OUT_DIR / "dataset2_parsed.csv", index=False)
enriched.to_csv(OUT_DIR / "dataset1_enriched.csv", index=False)

# Data dictionary for engineered fields
data_dict = pd.DataFrame({
    "column": [
        "habit_clean_raw", "habit_clean", "date",
        "minutes_after_sunset", "minutes_after_rat_arrival",
        "chronology_ok", "bat_landing_to_food_log1p", "bat_landing_to_food_sqrt",
        "season_label", "window_start", "window_end",
        "bat_landing_number", "food_availability", "rat_minutes", "rat_arrival_number"
    ],
    "description": [
        "Normalized habit string before rare-category consolidation",
        "Final habit category after synonym, order-invariant normalization, rare->Other",
        "Calendar date of landing",
        "Minutes from sunset to landing (derived from timestamps)",
        "Minutes from rat arrival to bat landing (derived from given seconds)",
        "True if start_time lies within [rat_period_start, rat_period_end]",
        "log1p transform of bat_landing_to_food for skew reduction",
        "sqrt transform of bat_landing_to_food for skew reduction",
        "Season string label (mapped from season/month)",
        "Start of matched 30-min observation window from dataset2",
        "End of matched 30-min observation window from dataset2",
        "Total bat landings in that 30-min interval (context/crowding)",
        "Estimated food availability in that interval",
        "Total minutes rats were present in that interval",
        "Rat arrival count in that interval (rat pressure)"
    ]
})
data_dict.to_csv(OUT_DIR / "engineered_data_dictionary.csv", index=False)

print("Written outputs to:", OUT_DIR.resolve())
